<h3>Preambula: </h3>

In [4]:
from collections.abc import Iterable

class Acceptor(object):
    '''
    Program realization of simple acceptor. 
    Since all instances are elements of set, they are hashable.
    
    Properties defined here:
        - sigma:set;
        - instances:set;
        - s0:object;
        - final_instances:set;
        - omega:dict
        
    You can get property like:
        instances:set = acceptor.instances
    But can not set property like:
        acceptor.sigma = {0, 1, 2}
        
    Methods defined here:
        accept(word) -> bool: returns is last instance for word is final;
    '''
    def __init__(self, sigma:set=None,
                       instances:set=None,
                       s0:object=None,
                       final_instances:set=None,
                       omega:dict=None):
        '''
        sigma - set of tokens (it is alphabet). 
        instances - set of instances.
        s0 - default instance. Should be in instances.
        final_instances - set of final instances. Should be subset of instances.
        omega - map for processing. Keys should be like (s, e), where:
            - s is in instances
            - e is in sigma
        Values of omega should be in instances.

        Also, s0 should be at list once in keys.
        '''
        
        # validate is sigma set 
        if not isinstance(sigma, set):
            raise TypeError('sigma argument is not of type set. Please, use set().')
            
        # validate is instances set
        if not isinstance(instances, set):
            raise TypeError('instances argument is not of type set. Please, use set().')
            
        # check is s0 in instances
        if s0 not in instances:
            raise ValueError('Default instance is not in instances set.')
            
        # check is final instances is set
        if not isinstance(final_instances, set):
            raise TypeError('final_instances is not of type set. Please, use set().')
            
        # check, is final instances subset of instances
        for instance in final_instances:
            if instance not in instances:
                raise ValueError(f'{instance} from final_instances is not in instances.')
            
        # check, is omega dict
        if not isinstance(omega, dict):
            raise TypeError('omega argument is not of type dict.')
            
        # if s0 is not in omega, acceptor can not work
        s0_in_omega:bool = False
            
        # keys of omega should be like (s, e)
        # values of omega should be in instances
        for key, value in omega.items():
            
            # key should be of type tuple
            if not isinstance(key, tuple):
                raise TypeError(f'{key} (type {type(key)}) is not of type tuple.')
                
            # key should be like (s, e)
            if len(key) != 2:
                raise TypeError(f'Keys in omega should be tuple with size 2, not {len(key)} ({key}).')
                
            # s should be in instances
            # e should be in sigma
            s, e = key
            
            if s not in instances:
                raise ValueError(f's in {key} is not in instances.')
                
            if e not in sigma:
                raise ValueError(f'e in {key} is not in sigma.')
                
            # value should be in instances
            if value not in instances:
                raise ValueError(f'{value} is not in sigma.')
                
            # search s0
            s0_in_omega:bool = (s == s0) or s0_in_omega

        # s0 is not in omega
        if not s0_in_omega:
            raise ValueError('s0 is not available in omega.')
                
        # create new instance of class
        self.__sigma:set = sigma
        self.__instances:set = instances
        self.__s0:object = s0
        self.__final_instances:set = final_instances
        self.__omega:dict = omega
            
            
    def accept(self, word:Iterable, recursive:bool=False) -> bool:
        '''
        Main method of acceptor. 
        Returns True, if last instance for word is final,
        else False.

        Uses two realizations, recursive and iterative.
        '''

        try:
            # convert to list for usability
            word:list = list(word)
        # word is not iterable
        except:
            raise TypeError('Argument in accept() is not iterable.')

        # recursive realization
        if recursive:
            def get_instance(cur_inst:object, word:list) -> object:
                '''
                Inner function, that gets last 
                '''
                # finish recursion
                if len(word) == 0:
                    return cur_inst
                else:
                    # get next instance
                    cur_inst:object = self.__omega[(cur_inst, word[0])]

                    # go to next token
                    return get_instance(cur_inst, word[1:])
        # iterative realization
        else:
            def get_instance(cur_inst:object, word:list) -> object:

                for token in word:
                    cur_inst:object = self.__omega[(cur_inst, token)]

                return cur_inst

        # get last instance
        last_inst:object = get_instance(self.__s0, word)

        return last_inst in self.__final_instances
       
    @property
    def sigma(self) -> set:
        '''
        Property to get sigma. Has not setter.
        '''
        return self.__sigma
    
    @property
    def instances(self) -> set:
        '''
        Property to get instances. Has not setter.
        '''
        return self.__instances
    
    @property
    def s0(self) -> object:
        '''
        Property to get s0. Has not setter.
        '''
        return self.__s0
    
    @property
    def final_instances(self) -> set:
        '''
        Property to get final_instances. Has not setter.
        '''
        return self.__final_instances
    
    @property
    def omega(self) -> set:
        '''
        Property to get omega. Has not setter
        '''
        return self.__omega

1. Build a determened acceptor that recognizes words belonging to the set specified by a regular expression (a * c | b * c) * over the alphabet = {a, b, c}. Justify each step of construction.

There are following components of acceptor:
$$\Sigma = (a, b, c)$$
$$S = (r_a, r_b, r_f, e)$$
$$s_o = r_f$$
$$F = (r)$$
$$\sigma:$$
$$\sigma(r_f, a) = r_a, \text{we started to read string like }a*$$
$$\sigma(r_f, b) = r_b, \text{we started to read string like }a*$$
$$\sigma(r_f, c) = r_f, \text{a* or b* can be empty strings}$$
$$\sigma(r_a, a) = r_a, \text{continue reading string like }a*$$
$$\sigma(r_a, c) = r_f, \text{string like }a* c \text{ was readed}$$
$$\sigma(r_a, b) = e, \text{error}$$
$$\sigma(r_b, b) = r_b, \text{continue reading string like } b*$$
$$\sigma(r_b, c) = r_f, \text{string like }b*c \text{ was readed}$$
$$\sigma(rb, a) = e, \text{error}$$
$$\text{Error instance can not change: }$$
$$\sigma(e, a) = e$$
$$\sigma(e, b) = e$$
$$\sigma(e, c) = e$$

In [5]:
# program realization
sigma:set = {'a', 'b', 'c'}
s:set = {'r_a', 'r_b', 'r_f', 'e'}
s0 = 'r_f'
f:set = {'r_f'}
omega:dict = {('r_a', 'a'):'r_a',
              ('r_a', 'c'):'r_f',
              ('r_a', 'b'):'e',
              ('r_b', 'a'):'e',
              ('r_b', 'b'):'r_b',
              ('r_b', 'c'):'r_f',
              ('r_f', 'a'):'r_a',
              ('r_f', 'b'):'r_b',
              ('r_f', 'c'):'r_f',
              ('e', 'a'):'e',
              ('e', 'b'):'e',
              ('e', 'c'):'e'}
    
# create acceptor
acceptor = Acceptor(sigma, s, s0, f, omega)

# testing
samples = ['bbbcbcab',
           'bbc',
           'aaacbcaccc',
           'bcacbcaacbb']

for sample in samples:
    print(f'{sample} - {acceptor.accept(sample)};')

bbbcbcab - False;
bbc - True;
aaacbcaccc - True;
bcacbcaacbb - False;


2. Show that the set $ (a ^ nb ^ na ^ n | n \ ge 0) $ is not regular.

An important consequence of the Mayhill-Nerode theorem is that a regular language can have a finite number of prefix equivalences. In this case, the number of classes is countable infinity, which contradicts the hypothesis of the theorem. So the given set is not regular.